# Taller 7

### SQL

##### Analítica Computacional para la Toma de Decisiones

---

|     Nombres      |      Apellidos       |     Login     |  Codigo   |
| :--------------: | :------------------: | :-----------: | :-------: |
|     Santiago     | Gonzalez Montealegre | s.gonzalez35  | 202012274 |
| Juliana Carolina |  Cardenas Barragan   | jc.cardenasb1 | 202011683 |

---


## 1. Consultas Básicas

### 1 
Cree un Jupyter notebook, el cual servirá como documento de trabajo y entrega. Incluya todas las respuestas a las consultas solicitadas en el notebook.

### 2
Conéctese al mismo servidor usado en las diapositivas, pero en este caso a la base de datos `taller_uni`.

In [1]:
import psycopg2
import pandas as pd

In [2]:
connection = psycopg2.connect(
    host="db-actd23-1.c2ejf18x6i7a.us-east-1.rds.amazonaws.com",
    port='5432',
    user="postgres",
    dbname="taller_uni",
    password="postgres",
)

Se definirán funciónes con el fin de facilitar las consultas y no tener que repetir código. Además, facilitar la interpretación de las consultas.

In [12]:
# ------------------------------------------------
# Mostrar el resultado como un pandas DataFrame
# ------------------------------------------------

def getCursor(connection)->(psycopg2.extensions.cursor):
    cursor = connection.cursor()
    return cursor

# Ejecutar la query
def execute(query: str, cursor: (psycopg2.extensions.cursor)):
    cursor.execute(query)
    result = cursor.fetchall()
    return result

# Hallar los atributos
def getColumnNames(cursor):
    columns = [i[0] for i in cursor.description]
    return columns

# Mostrar como DataFrame
def pdsql(query: str, connection: (psycopg2.extensions.connection)):
    try:
        cursor = connection.cursor()
        result = execute(query, cursor)
        columns = getColumnNames(cursor)
        df = pd.DataFrame(result, columns=columns)
        return df
    except Exception:
        try:
            connection = psycopg2.connect(
                host="db-actd23-1.c2ejf18x6i7a.us-east-1.rds.amazonaws.com",
                port='5432',
                user="postgres",
                dbname="taller_uni",
                password="postgres",
            )
            cursor = connection.cursor()
            result = execute(query, cursor)
            columns = getColumnNames(cursor)
            df = pd.DataFrame(result, columns=columns)
            return df
        except:
            return "Query Error"

### 3
Liste todas las tablas en el esquema público.

In [4]:
query = """
SELECT * 
FROM pg_catalog.pg_tables 
WHERE schemaname='public'
;"""

df = pdsql(query, connection)
df[["tablename"]]

,tablename
0,unid_acad
1,curso
2,instructor
3,grupo
4,dicta


### 4
Liste el nombre de todas las unidades académicas.

In [5]:
query = """
SELECT *
FROM unid_acad
;"""
df = pdsql(query, connection)
df[["nombre_unid"]]

,nombre_unid
0,IIND
1,ECON
2,BIOL


### 5
Liste los nombres de todas las unidades académicas a las que están asociados los profesores. Elimine duplicados con `distinct` antes del atributo de interés (nombre de la unidad).


In [6]:
query = """
SELECT DISTINCT unid_acad.nombre_unid
FROM unid_acad
NATURAL INNER JOIN instructor
;"""
df = pdsql(query, connection)
df

,nombre_unid
0,IIND
1,ECON


### 6
Para cada tabla liste 5 registros usando `limit` al final de la consulta.

In [7]:
query = """
SELECT *
FROM unid_acad
LIMIT 5
;"""
df = pdsql(query, connection)
df

,nombre_unid,edificio,presupuesto
0,IIND,ML,100.00
1,ECON,W,800.00
2,BIOL,A,500.00


In [8]:
query = """
SELECT *
FROM curso
LIMIT 5
;"""
df = pdsql(query, connection)
df

,curso_cod,nombre,nombre_unid,creditos
0,000,probyest1,IIND,3
1,001,anadec,IIND,3
2,002,optimizacion,IIND,3
3,003,micro1,ECON,3


In [9]:
query = """
SELECT *
FROM instructor
LIMIT 5
;"""
df = pdsql(query, connection)
df

,inst_id,nombre,nombre_unid,salario
0,1,Pedro Perez,IIND,100.00
1,2,Ana Alferez,IIND,200.00
2,3,Camila Cifuentes,ECON,500.00
3,4,Reinaldo Rondon,IIND,300.00


In [10]:
query = """
SELECT *
FROM grupo
LIMIT 5
;"""
df = pdsql(query, connection)
df

,curso_cod,grupo_cod,semestre,ano
0,000,1,1,2023
1,001,1,2,2023
2,002,1,1,2023
3,002,2,1,2023


In [11]:
query = """
SELECT *
FROM dicta
LIMIT 5
;"""
df = pdsql(query, connection)
df

,inst_id,curso_cod,grupo_cod,semestre,ano
0,1,000,1,1,2023
1,2,001,1,2,2023
2,3,002,1,1,2023
3,4,002,1,1,2023


### 7
Realice una consulta que retorne un listado de los `instructores` con su nombre, ID y el doble de su salario.

In [15]:
query = """
SELECT nombre, inst_id, 2 * salario as salarioX2
FROM instructor
;"""
df = pdsql(query, connection)
df

,nombre,inst_id,salariox2
0,Pedro Perez,1,200.00
1,Ana Alferez,2,400.00
2,Camila Cifuentes,3,1000.00
3,Reinaldo Rondon,4,600.00


### 8
Realice una consulta para obtener todos los cursos ofrecidos por la unidad `'IIND'`.

In [16]:
query = """
SELECT *
FROM curso
WHERE nombre_unid = 'IIND'
;"""
df = pdsql(query, connection)
df

,curso_cod,nombre,nombre_unid,creditos
0,000,probyest1,IIND,3
1,001,anadec,IIND,3
2,002,optimizacion,IIND,3
